# Harmonised API examples the EUM / ESA / CMEMS data

In [1]:
import os
import sys
import json
import time
from zipfile import ZipFile
sys.path.append(os.path.dirname(os.getcwd()) + '/Hub_Tools/')
import harmonised_data_access_api_tools as hapi

In [2]:
t0 = time.time()

In [3]:
api_key = "cmJ1UGJQVzZnT09HU2RUWDJhTGFkOGY4RjhnYTpGRmFCTTNoSXluVk1NdEk4b2dPc2ZjMHFOdlVh"
download_dir_path = "/home/jovyan/work/test_data"
verbose = False

In [4]:
# example data sets available: codes from here >> https://www.wekeo.eu/dataset-navigator/start

# ---------------------------- CMEMS options --------------------------
#dataset_id = "EO:MO:DAT:SEAICE_GLO_SEAICE_L4_NRT_OBSERVATIONS_011_001" # tested
#dataset_id = "EO:MO:DAT:OCEANCOLOUR_ARC_CHL_L3_REP_OBSERVATIONS_009_069" # ESA OC-CCI REP

# ---------------------------- C3S options ----------------------------
#dataset_id = "EO:ECMWF:DAT:ERA5_HOURLY_DATA_ON_SINGLE_LEVELS_1979_PRESENT" # ERA5

# ---------------------------- CAMS options ---------------------------
#dataset_id = "EO:ECMWF:DAT:CAMS_SOLAR_RADIATION_TIMESERIES" # SOLAR RAD

# ----------------- EUMETSAT COPERNICUS MARINE options ----------------
#dataset_id = "EO:EUM:DAT:SENTINEL-3:SR_1_SRA___" # SRAL L1B tested
#dataset_id = "EO:EUM:DAT:SENTINEL-3:SR_1_SRA_BS___" #SRAL L1BS tested
#dataset_id = "EO:EUM:DAT:SENTINEL-3:SR_1_SRA_A__" #SRAL L1A >>>> not available <<<<
#dataset_id = "EO:EUM:DAT:SENTINEL-3:SR_2_WAT___" # SRAL L2 tested
#dataset_id = "EO:EUM:DAT:SENTINEL-3:OL_1_ERR___" # OLCI ERR tested
#dataset_id = "EO:EUM:DAT:SENTINEL-3:OL_1_EFR___" # OLCI EFR tested
#dataset_id = "EO:EUM:DAT:SENTINEL-3:OL_2_WRR___" # OLCI WRR tested
#dataset_id = "EO:EUM:DAT:SENTINEL-3:OL_2_WFR___" # OLCI WFR tested
#dataset_id = "EO:EUM:DAT:SENTINEL-3:SL_1_RBT___" # SLSTR L1 RBT tested
#dataset_id = "EO:EUM:DAT:SENTINEL-3:SL_2_WST___" # SLSTR L2 WST tested

# ------------------------- EUMETSAT other options -------------------
#dataset_id = "EO:EUM:SV:EUMETSAT:V01"
                
# ------------------------- ESA MARINE options ------------------------
#dataset_id = "EO:ESA:DAT:SENTINEL-2:MSI1C"       # MSI L1C tested
#dataset_id = "EO:ESA:DAT:SENTINEL-2:MSI2A"       # MSI L2A not available >>>> not available <<<<
#dataset_id = "EO:ESA:DAT:SENTINEL-1:L1_GRD"      # S1 L1 GRD tested
#dataset_id = "EO:ESA:DAT:SENTINEL-1:L1_SLC"      # S1 L1 SLC tested

Query file /home/jovyan/work/ocean-wekeo-jpyhub/JSON_templates/EO_EUM_DAT_SENTINEL-3_OL_2_WRR___.json does not exist
Script will stop after showing metadata, to aid in creating a query file.


In [ ]:
# find query file
JSON_query_file = os.path.join(os.path.dirname(os.getcwd()),"JSON_templates",dataset_id.replace(':','_')+".json")
if not os.path.exists(JSON_query_file):
    print('Query file ' + JSON_query_file + ' does not exist')
    print('Script will stop after showing metadata, to aid in creating a query file.')
    need_meta = True
else:
    print('Found JSON query file, you may want to adapt it.')
    need_meta = False

In [5]:
HAPI_dict = hapi.init(dataset_id, api_key, download_dir_path, verbose=verbose)
HAPI_dict = hapi.get_access_token(HAPI_dict)
HAPI_dict = hapi.accept_TandC(HAPI_dict)

# check meta data for the dataset product >> query generation
if need_meta:
    HAPI_dict = hapi.query_metadata(HAPI_dict)
    sys.exit()
    
# load the query
with open(JSON_query_file, 'r') as f:
    query = json.load(f)

print('--------------------------------')
print('Elapsed time: %s' % (time.time() - t0))
print('--------------------------------')

Getting an access token. This token is valid for one hour only.
Success: Access token is 155f472b-4206-34bf-a993-649335d1552e
Copernicus_General_License Terms and Conditions already accepted
Getting query metadata, URL Is https://apis.wekeo.eu/databroker/0.1.0/querymetadata/EO%3AEUM%3ADAT%3ASENTINEL-3%3AOL_2_WRR___?access_token=155f472b-4206-34bf-a993-649335d1552e
************** Query Metadata for EO:EUM:DAT:SENTINEL-3:OL_2_WRR___ **************
{
    "datasetId": "OL_2_WRR___",
    "parameters": {
        "boundingBoxes": [
            {
                "comment": "What area of the Earth would you like your data to cover?",
                "details": {
                    "crs": "EPSG:4326",
                    "extent": [
                        -180.0,
                        -90.0,
                        180.0,
                        90.0
                    ]
                },
                "label": "Select a geographic extent",
                "name": "bbox",
               

SystemExit: 

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# launch job
HAPI_dict = hapi.launch_query(HAPI_dict, query)

# wait for jobs to complete
hapi.check_job_status(HAPI_dict)

# check results
HAPI_dict = hapi.get_results_list(HAPI_dict)
HAPI_dict = hapi.get_download_links(HAPI_dict)

# download data
HAPI_dict = hapi.download_data(HAPI_dict, skip_existing=True)
print('--------------------------------')
print('Elapsed time: %s' % (time.time() - t0))
print('--------------------------------')